In [2]:
import os, sys
import pandas as pd
import numpy as np
import re
from googletrans import Translator
import sqlalchemy
from tqdm import tqdm, trange
from scipy.stats.stats import pearsonr, spearmanr  
from scipy.stats import zscore
import random
import pingouin as pg
import math
from sklearn.preprocessing import normalize

### Read in relevant files/MySQL tables

In [3]:
db_twitter = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='twitterSuperUsers', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
db_lexica = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='dlatk_lexica', query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})
db_county = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='county_data', query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})
db_project = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='individualism_collectivism', query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})

engine = sqlalchemy.create_engine(db_project)
# county_scores = pd.read_sql("feat$cat_individVsCollectFinal$msgs_100u$cnty$1gra", con=engine)
# state_scores = pd.read_sql("feat$cat_individVsCollectFinal$msgs_100u$state$1gra", con=engine)

county_scores = pd.read_sql("feat$cat_individVsCollect_w$msgs_100u$cnty$1gra", con=engine)
state_scores = pd.read_sql("feat$cat_individVsCollect_w$msgs_100u$state$1gra", con=engine)

engine = sqlalchemy.create_engine(db_lexica)
lexicon = pd.read_sql("individVsCollectFinal", con=engine)

engine = sqlalchemy.create_engine(db_county)
county_mapping = pd.read_sql("county_by_state_reg_div", con=engine)
community_mapping = pd.read_sql("superCrossWalk_FINAL_FINAL", con=engine)

/home/shreyah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  """Entry point for launching an IPython kernel.
/home/shreyah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  
/home/shreyah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shreyah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SADeprecationWarning: Calling URL() directly is deprecated and will be dis

### Utils

In [4]:
#go from county name --> fips code
def name_to_county(name):
    #get state name
    state = name.split(",")[-1].strip()
    if(state == "Puerto Rico"): 
        return None
    #get county name
    county = (name.split(",")[0]).split("County")[0].strip()
    county = county.split("Municipality")[0].strip()
    county = county.split("Borough")[0].strip()
    county = county.split("Parish")[0].strip()
    county = county.replace("city", "City")
    county = county.replace("ñ", "n")
   
    temp = county_mapping[county_mapping["state"] == state]
    temp = temp[temp["county"] == county]["cnty"]
    try:
        return np.array(temp)[0]
    except:
        print("Couldn't map to fips code: ", name)
        return None

#calculate correlation between two county-level data points
def calculate_correlation(data1, data2):
    overlapping_keys = np.intersect1d(list(data1.keys()), list(data2.keys()))
    values1 = [data1[x] for x in overlapping_keys]
    values2 = [data2[x] for x in overlapping_keys]
    return pearsonr(values1, values2)

def get_county_map():
    mapping = {}
    with open("GCI Data/counties.txt") as f:
        lines = [line.rstrip() for line in f]
        for l in lines:
            parts = l.split(",")
            mapping[parts[1]] = int(parts[0])
    return mapping

#get state-aggregated data for county-level data (average over all counties)
def get_state_data(county_data):
    mapping = get_county_map()
    state_data = {}
    for c in county_data:
        try:
            state = list((county_mapping[county_mapping["cnty"] == c])["state"])[0]
        except:
            continue
        if state not in state_data.keys():
            state_data[state] = []
        state_data[state].append(county_data[c])       
    
    state_data_final = {}
    for s in state_data:
        state_data_final[s] = np.average(state_data[s])
    return normalize_data(state_data_final)

def normalize_data(data):
    keys = data.keys()
    vals = np.array([data[x] for x in keys])
    normalized_vals = zscore(vals) 
    return dict(zip(keys,normalized_vals))

#get correlation between single (list) and (list of lists) at the county level
def validate_data(input_data, other_data):
    overlapping_keys = list(input_data.keys())
    for d in other_data:
        overlapping_keys = np.intersect1d(list(d.keys()), overlapping_keys)
    
    data_values = np.array([input_data[x] for x in overlapping_keys])
    other_data_intersect = []
    for d in other_data:
        other_data_intersect.append(np.array([d[x] for x in overlapping_keys]))   
    print(pearsonr(data_values, sum(other_data_intersect)))   

def get_overlapping_keys(dicts):
    overlapping_keys = list(dicts[0].keys())
    for d in dicts:
        overlapping_keys = np.intersect1d(list(d.keys()), overlapping_keys)
    return overlapping_keys

def process_dicts(dicts):
    keys = get_overlapping_keys(dicts)
    processed_lists = []   
    for d in dicts:
        processed_list = np.array([d[x] for x in keys])
        processed_lists.append(processed_list)
    return keys, processed_lists

def get_community_mapping():
    cnty = community_mapping["cnty"]
    ACP_name = community_mapping["ACP_name"]
    mappings = {}   
    communities = np.array(ACP_name)
    communities = np.unique(communities[communities != np.array(None)])
    for i in range(len(cnty)):      
        mappings[cnty[i]] = ACP_name[i]
    return communities, mappings

def zero_one_normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
    

In [5]:
income_data = pd.read_csv("GCI Data/income_census.csv")
income_data.set_index("Label (Grouping)", inplace=True)
income_data = income_data.T

income_string = "Median earnings (dollars) for full-time, year-round workers with earnings"

#remove unecessary data
temp_columns = income_data.columns
for c in temp_columns:
    if (income_string in c):
        income_data["income"] = income_data[c]
income_data.drop(columns=temp_columns, inplace=True)

income_control = {}
for index, row in income_data.iterrows():
    if("Total!!Estimate" not in index): continue
    name = index.split("!!")[0]
    fips = name_to_county(name)
    if fips is not None:                       
        try:
            income = int(row["income"].replace(",", ""))
            income_control[fips] = income
        except:
            continue
income_control = normalize_data(income_control)

Couldn't map to fips code:  United States


### Indicator 1: Fertility Rate

##### Total fertility rate for each county, US Census, county level

In [6]:
# fertility_data = pd.read_csv("GCI Data/fertility_census.csv")
# fertility_data.set_index("Label (Grouping)", inplace=True)
# rate_string = "!!Women with births in the past 12 months !!Rate per 1,000 women!!Estimate"

# #remove unecessary data
# for c in fertility_data.columns:
#     if(rate_string in c):
#         name = c.split(rate_string)[0]
#         fertility_data[name] = fertility_data[c]
#     fertility_data.drop(columns=[c], inplace=True)

# fertility_data = fertility_data.T
# columns_temp = fertility_data.columns

# #keep only relevant columns
# fertility_data["all"] = fertility_data["Women 15 to 50 years"]
# fertility_data["15-19"] = fertility_data["15 to 19 years"]
# fertility_data["20-34"] = fertility_data["20 to 34 years"]
# fertility_data["35-50"] = fertility_data["35 to 50 years"]
# fertility_data.drop(columns=columns_temp, inplace=True)

In [7]:
# def get_tfr(row):
#     try:
#         return (int(row["15-19"])*5 + int(row["20-34"])*15 + int(row["35-50"])*16)/1000
#     except:
#         return None

# tfr_data = {}
# for index,row in fertility_data.iterrows():
#     tfr = get_tfr(row)
#     fips = name_to_county(index)
#     if(tfr is not None and fips is not None):
#         tfr_data[fips] = tfr

# tfr_data = normalize_data(tfr_data)

### Indicator 2: Living Arrangements

##### Number of households with grandparents living with grandchildren, US Census, county level

In [8]:
household_data = pd.read_csv("GCI Data/household_census.csv")
household_data.set_index("Label (Grouping)", inplace=True)
household_data = household_data.T

# coll_string = "Household with grandparents living with grandchildren:"
coll_string = "Household with grandparent responsible for own grandchildren under 18 years"
other_string = "Household without grandparents living with grandchildren"

#remove unecessary data
temp_columns = household_data.columns
temp_columns = temp_columns.drop("Total:")
for c in temp_columns:
    if (coll_string in c):
        household_data["total_coll"] = household_data[c]
    elif (other_string in c):
        household_data["total_other"] = household_data[c]
household_data.drop(columns=temp_columns, inplace=True)

living_data = {}
for index, row in household_data.iterrows():
    name = index.split("!!")[0]
    fips = name_to_county(name)
    if fips is not None:                       
        try:
            # total = int(row["Total:"].replace(",", ""))
            total = int(row["total_coll"].replace(",", "")) + int(row["total_other"].replace(",", ""))
            total_coll = int(row["total_coll"].replace(",", ""))
            living_data[fips] = (total_coll)/total
        except:
            continue

living_data = normalize_data(living_data)

Couldn't map to fips code:  United States


### Indicator 3: Stability of Marriage

##### Ratio of married individuals to divorced individuals 15 years and over (sum of men and women), US Census, county level

In [9]:
# marital_data = pd.read_csv("GCI Data/divorce_census.csv")
# marital_data.set_index("Label (Grouping)", inplace=True)
# marital_data = marital_data.T

# total_string_male = "Males 15 years and over"
# total_string_female = "Females 15 years and over"
# divorce_string = "Divorced"
# separated_string = "Separated"
# married_string = "Now married, except separated"

# #remove unecessary data
# temp_columns = marital_data.columns
# for c in temp_columns:
#     if(divorce_string in c):
#         label = (c.split("("))[1].split(")")[0]
#         marital_data["num_divorced_"+label] = marital_data[c]
#     elif(married_string in c):
#         label = (c.split("("))[1].split(")")[0]
#         marital_data["num_married_"+label] = marital_data[c]
#     elif(separated_string in c):
#         label = (c.split("("))[1].split(")")[0]
#         marital_data["num_separated_"+label] = marital_data[c]
#     elif(total_string_male in c):
#         marital_data["total_male"] = marital_data[c]
#     elif(total_string_female in c):
#         marital_data["total_female"] = marital_data[c]
# marital_data.drop(columns=temp_columns, inplace=True)

# marriage_data = {}
# for index, row in marital_data.iterrows():
#     if("Percent" in index): continue
#     name = index.split("!!")[0]
#     fips = name_to_county(name)
#     if fips is not None:                       
#         try:
#             married = int(row["num_married_male"].replace(",", "")) + int(row["num_married_female"].replace(",", ""))
#             divorced = int(row["num_divorced_male"].replace(",", "")) + int(row["num_divorced_female"].replace(",", "")) 
#             separated = int(row["num_separated_male"].replace(",", "")) + int(row["num_separated_female"].replace(",", ""))
#             total = int(row["total_male"].replace(",", "")) + int(row["total_female"].replace(",", ""))
#             marriage_data[fips] = married/divorced
#         except:
#             continue

# marriage_data = normalize_data(marriage_data)

### Indicator 4: Religiosity

##### Score for question "For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all? – Religion" (scale of 1-4) 
##### Inverse weighted so high importance --> high score, WVS Survey, state level
##### Standard devation per state = 1.02 --> enforcing that each state must have > 10 responses

In [10]:
wvs_data = pd.read_csv("GCI Data/WVS_Cross-National_Wave_7.csv")
wvs_data_usa = wvs_data[wvs_data["B_COUNTRY_ALPHA"] == "USA"]

wvs_state_mapping = {}
with open("GCI Data/wvs_states.txt") as f:
    lines = [line.rstrip() for line in f]
for l in lines:
    parts = l.split(" ")
    wvs_state_mapping[int(parts[0])] = " ".join(parts[3:])    

/home/shreyah/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (345,347,349,574,575,576) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
religion_data_temp = {}

for index, row in wvs_data_usa.iterrows():
    state = wvs_state_mapping[row["N_REGION_WVS"]]
    religious_importance = row["Q6"]
    if(religious_importance > 0):
        if(state not in religion_data_temp.keys()):
            religion_data_temp[state] = []
        religion_data_temp[state].append(religious_importance)

religion_data = {}
std_devs = []
for s in religion_data_temp:
    if(len(religion_data_temp[s]) > 10):
        religion_data[s] = 1/np.average(religion_data_temp[s])
    std_devs.append(np.std(religion_data_temp[s]))
print("STD DEV:", np.average(std_devs))
    
religion_data = normalize_data(religion_data)

STD DEV: 1.0217751798668622


### Indicator 5: Collective Transportation

##### Average number of cars per household, inverse weights so high number of cars --> low score, US Census, county level

In [12]:
# transportation_data = pd.read_csv("GCI Data/transportation_census.csv")
# transportation_data.set_index("Label (Grouping)", inplace=True)
# transportation_data = transportation_data.T

# temp_columns = transportation_data.columns
# temp_columns = temp_columns.drop("Total:")
# for c in temp_columns:
#     col_name = c.strip()
#     transportation_data[col_name] = transportation_data[c]
# transportation_data.drop(columns=temp_columns, inplace=True)

# def get_cars_per_household(row):
#     total = int(row["Total:"].replace(",", ""))
#     none = int(row["No vehicle available"].replace(",", ""))
#     car1 = int(row["1 vehicle available"].replace(",", ""))
#     car2 = int(row["2 vehicles available"].replace(",", ""))
#     car3 = int(row["3 vehicles available"].replace(",", ""))
#     car4 = int(row["4 or more vehicles available"].replace(",", ""))
#     return (car1 + car2*2 + car3*3 + car4*4)/total

# car_data = {}
# for index, row in transportation_data.iterrows():
#     name = index.split("!!")[0]
#     fips = name_to_county(name)
#     if fips is not None:                       
#         try:
#             car_data[fips] = 1/get_cars_per_household(row)
#         except:
#             continue
# car_data = normalize_data(car_data)

### Indicator 6: Ingroup Bias

##### Score for question "Do you agree, disagree or neither agree nor disagree with the following statements? - When jobs are scarce, employers should give priority to people of this country over immigrants." (scale of 1-5) 
##### Inverse weighted so high agreement --> high score, WVS Survey, state level
##### Average standard devation per state = 0.82 --> enforcing each state must have > 5 responses

In [13]:
compatriotism_data_temp = {}

for index, row in wvs_data_usa.iterrows():
    state = wvs_state_mapping[row["N_REGION_WVS"]]
    bias = row["Q34"]
    if(bias > 0):
        if(state not in compatriotism_data_temp.keys()):
            compatriotism_data_temp[state] = []
        compatriotism_data_temp[state].append(bias)

compatriotism_data = {}
std_devs = []
for s in compatriotism_data_temp:
    compatriotism_data[s] = 1/np.average(compatriotism_data_temp[s]) 
    std_devs.append(np.std(compatriotism_data_temp[s]))
print("STD DEV:", np.average(std_devs))

compatriotism_data = normalize_data(compatriotism_data)

STD DEV: 0.8247487844563858


### Aggregate Data

##### All data at the state level

In [14]:
# tfr_data_state = get_state_data(tfr_data)
living_data_state = get_state_data(living_data)
# marriage_data_state = get_state_data(marriage_data)
# car_data_state = get_state_data(car_data)
income_data_state = get_state_data(income_control)

# state_data = [tfr_data_state, living_data_state, marriage_data_state, car_data_state, religion_data, compatriotism_data]
state_data = [living_data_state, income_data_state, religion_data, compatriotism_data]
# state_labels = ["fertility", "grandparents", "marriage", "cars", "religion", "compatriotism"]
state_labels = ["grandparents", "income", "religion", "compatriotism"]
#make state df
fips_codes, state_data_processed = process_dicts(state_data)
state_df = pd.DataFrame(columns=state_labels)
for i in range(len(state_labels)):
    state_df[state_labels[i]] = state_data_processed[i]
state_df.set_index(fips_codes, inplace=True)

print("state df shape: ",  state_df.shape)
print("Pairwise Pearson Correlations at the state level (all)")
display(state_df.corr())
print("Cronbach alpha")
print(pg.cronbach_alpha(data=state_df))

state df shape:  (42, 4)
Pairwise Pearson Correlations at the state level (all)


,grandparents,income,religion,compatriotism
grandparents,1.000000,-0.538756,0.549095,0.317850
income,-0.538756,1.000000,-0.413523,-0.180937
religion,0.549095,-0.413523,1.000000,0.749607
compatriotism,0.317850,-0.180937,0.749607,1.000000


Cronbach alpha
(0.33480090412993235, array([-0.068,  0.611]))


##### Ran experiments with every subset of >3 variables at the state level to maximize Cronbach's Alpha. Result: Include living arrangements, religiosity, and compatriotism

In [15]:

# living_data_state = get_state_data(living_data)
# income_data_state = get_state_data(income_control)

# state_data = [living_data_state, religion_data, compatriotism_data]
# state_labels = ["grandparents", "religion", "compatriotism"]

# # state_data = [tfr_data_state, living_data_state, religion_data, compatriotism_data]
# # state_labels = ["fertility", "grandparents", "religion", "compatriotism"]

# #make state df
# fips_codes, state_data_processed = process_dicts(state_data)
# state_df = pd.DataFrame(columns=state_labels)
# for i in range(len(state_labels)):
#     state_df[state_labels[i]] = state_data_processed[i]
# state_df.set_index(fips_codes, inplace=True)

# print("state df shape: ",  state_df.shape)
# print("Pairwise Pearson Correlations at the state level")
# display(state_df.corr())
# print("Cronbach alpha")
# print(pg.cronbach_alpha(data=state_df))

# state_data = [living_data_state, religion_data, compatriotism_data, income_data_state]
# state_labels = ["grandparents", "religion", "compatriotism", "income"]
# #make state df
# fips_codes, state_data_processed = process_dicts(state_data)
# state_df = pd.DataFrame(columns=state_labels)
# for i in range(len(state_labels)):
#     state_df[state_labels[i]] = state_data_processed[i]
# state_df.set_index(fips_codes, inplace=True)

# # print("Pairwise Pearson Correlations at the state level (including income)")
# # display(state_df.corr())
# state_df.corr().to_csv("test.csv")
# print("Partial correlations controlling for income")
# display(state_df.pcorr())


### Looking at Collectivism/Individualism Scores: County level

In [16]:
# coll_scores_county = county_scores[county_scores["feat"] == "COLLECTIVIST"]
# coll_scores_county = dict(zip(coll_scores_county.group_id, coll_scores_county.group_norm))

# indv_scores_county = county_scores[county_scores["feat"] == "INDIVIDUALIST"]
# indv_scores_county = dict(zip(indv_scores_county.group_id, indv_scores_county.group_norm))

# county_data=[living_data, marriage_data, car_data, coll_scores_county, indv_scores_county, income_control]
# county_labels = ["fertility", "grandparents", "marriage", "cars", "collectivism", "individualism", "income"]

# #make county df
# fips_codes, state_data_processed = process_dicts(county_data)
# county_df = pd.DataFrame(columns=county_labels)
# for i in range(len(county_labels)):
#     county_df[county_labels[i]] = state_data_processed[i]
# county_df.set_index(fips_codes, inplace=True)

# print("county df shape: ", county_df.shape)
# print("Pairwise Pearson Correlations at the county level (all)")
# display(county_df.corr())
# print("Cronbach alpha")
# print(pg.cronbach_alpha(data=county_df))

# print(len(county_df))


In [17]:
from scipy.stats import pearsonr


#Table names
# feat$cat_ablation_045_075_w$msgs_100u$cnty$1gra
# feat$cat_ablation_045_07_w$msgs_100u$cnty$1gra
# feat$cat_ablation_045_08_w$msgs_100u$cnty$1gra
# feat$cat_ablation_04_075_w$msgs_100u$cnty$1gra
# feat$cat_ablation_04_07_w$msgs_100u$cnty$1gra
# feat$cat_ablation_04_08_w$msgs_100u$cnty$1gra
# feat$cat_ablation_05_075_w$msgs_100u$cnty$1gra
# feat$cat_ablation_05_07_w$msgs_100u$cnty$1gra
# feat$cat_ablation_05_08_w$msgs_100u$cnty$1gra

TABLE_NAMES = ["feat$cat_ablation_045_075_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_045_07_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_045_08_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_04_075_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_04_07_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_04_08_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_05_075_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_05_07_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_05_08_w$msgs_100u$cnty$1gra",
                "feat$cat_ablation_1_1_w$msgs_100u$cnty$1gra"]

purification_param = "indv"
PURIFICATION_NAMES = ["feat$cat_purification_{}_0_01_w$msgs_100u$cnty$1gra".format(purification_param),
                        "feat$cat_purification_{}_0_05_w$msgs_100u$cnty$1gra".format(purification_param),
                        "feat$cat_purification_{}_0_1_w$msgs_100u$cnty$1gra".format(purification_param),
                        "feat$cat_purification_{}_0_15_w$msgs_100u$cnty$1gra".format(purification_param),
                        "feat$cat_purification_{}_0_2_w$msgs_100u$cnty$1gra".format(purification_param),
                        "feat$cat_purification_{}_0_25_w$msgs_100u$cnty$1gra".format(purification_param)]

table_labels = ["scores_final", "scores_seed_only", "scores_expansion_only"]
table_names = ["feat$cat_purification_015_w$msgs_100u$cnty$1gra", "feat$cat_ablation_1_1_w$msgs_100u$cnty$1gra", "feat$cat_ablation_045_075_w$msgs_100u$cnty$1gra"]


def print_val_data(table_name, label):
    engine = sqlalchemy.create_engine(db_project)
    county_scores = pd.read_sql(table_name, con=engine)
    # county_scores = pd.concat([pd.read_sql(table_name, con=engine), pd.read_sql(table_name.replace("indv", "coll"), con=engine)])

    # county_scores = pd.read_sql("feat$cat_individVsCollect_w$msgs_100u$cnty$1gra", con=engine)
    # state_scores = pd.read_sql("feat$cat_individVsCollect_w$msgs_100u$state$1gra", con=engine)

    coll_scores_county = county_scores[county_scores["feat"] == "COLLECTIVISM"]
    if(len(coll_scores_county) == 0):
        coll_scores_county = county_scores[county_scores["feat"] == "COLLECTIVIST"]
    coll_scores_county = dict(zip(coll_scores_county.group_id, coll_scores_county.group_norm))

    indv_scores_county = county_scores[county_scores["feat"] == "INDIVIDUALISM"]
    if(len(indv_scores_county) == 0):
        indv_scores_county = county_scores[county_scores["feat"] == "INDIVIDUALIST"]
    indv_scores_county = dict(zip(indv_scores_county.group_id, indv_scores_county.group_norm))


    coll_scores_state = get_state_data(coll_scores_county)
    indv_scores_state = get_state_data(indv_scores_county)
    
    living_data_state = get_state_data(living_data)
    income_data_state = get_state_data(income_control)

    state_data = [living_data_state, religion_data, compatriotism_data, income_data_state, coll_scores_state, indv_scores_state]
    coll_label = "{}_coll".format(label)
    indv_label = "{}_indv".format(label)
    state_labels = ["grandparents", "religion", "compatriotism", "income", "collectivism", "individualism"]

    #make state df
    states, state_data_processed = process_dicts(state_data)
    state_df = pd.DataFrame(columns=state_labels)
    for i in range(len(state_labels)):
        state_df[state_labels[i]] = zero_one_normalize(state_data_processed[i])
    state_df.set_index(states, inplace=True)

    def get_corr(col1, col2):
        return pearsonr(state_df[col1], state_df[col2])

    grandparents_indv =  get_corr("individualism", "grandparents")[0]
    religion_indv = get_corr("individualism", "religion")[0]
    compatriotism_indv = get_corr("individualism", "compatriotism")[0]
    grandparents_coll = get_corr("collectivism", "grandparents")[0]
    religion_coll = get_corr("collectivism", "religion")[0]
    compatriotism_coll = get_corr("collectivism", "compatriotism")[0]

    grandparents_indv_pval = get_corr("individualism", "grandparents")[1]
    religion_indv_pval = get_corr("individualism", "religion")[1]
    compatriotism_indv_pval = get_corr("individualism", "compatriotism")[1]
    grandparents_coll_pval = get_corr("collectivism", "grandparents")[1]
    religion_coll_pval = get_corr("collectivism", "religion")[1]
    compatriotism_coll_pval = get_corr("collectivism", "compatriotism")[1]

    indv_running_average = (get_corr("individualism", "grandparents")[0] + 
                    get_corr("individualism", "religion")[0] + 
                    get_corr("individualism", "compatriotism")[0])
    
    coll_running_average = (get_corr("collectivism", "grandparents")[0] +
                            get_corr("collectivism", "religion")[0] +
                            get_corr("collectivism", "compatriotism")[0])

    vc_scores = pd.read_csv("GCI Data/vandello_cohen.csv")
    vc_scores = dict(zip(vc_scores.State, vc_scores.Score))

    state_data = [vc_scores, coll_scores_state, indv_scores_state]
    state_labels = ["vandello-cohen", "collectivism", "individualism"]

    #make state df
    states, state_data_processed = process_dicts(state_data)
    state_df = pd.DataFrame(columns=state_labels)
    for i in range(len(state_labels)):
        state_df[state_labels[i]] = zero_one_normalize(state_data_processed[i])
    state_df.set_index(states, inplace=True)

    indv_vc = get_corr("individualism", "vandello-cohen")[0]
    coll_vc = get_corr("collectivism", "vandello-cohen")[0]

    indv_vc_pval = get_corr("individualism", "vandello-cohen")[1]
    coll_vc_pval = get_corr("collectivism", "vandello-cohen")[1]

    average_corr_indv = (indv_running_average + get_corr("vandello-cohen", "individualism")[0])/4
    average_corr_coll = (coll_running_average + get_corr("vandello-cohen", "collectivism")[0])/4

    #print latex table wither rounding to 3 dec points
    #individualism: vc, grandparents, religion, compatriotism
    #print correlations, add a * if p < 0.05
    def get_string(corr, pval):
        if(pval < 0.05): return str(round(corr, 3)) + "$^*$"
        else: return str(round(corr, 3))
    
    print(get_string(indv_vc, indv_vc_pval), "&", get_string(grandparents_indv, grandparents_indv_pval), "&", get_string(religion_indv, religion_indv_pval), "&", get_string(compatriotism_indv, compatriotism_indv_pval), "&", get_string(average_corr_indv, 0), "\\\\")
    print(get_string(coll_vc, coll_vc_pval), "&", get_string(grandparents_coll, grandparents_coll_pval), "&", get_string(religion_coll, religion_coll_pval), "&", get_string(compatriotism_coll, compatriotism_coll_pval), "&", get_string(average_corr_coll, 0), "\\\\")

for t in TABLE_NAMES:
    print(t)
    print_val_data(t, t)
    print()


# state_df_csv = pd.DataFrame()
# state_df_vc_csv = pd.DataFrame()
# for t,label in zip(table_names, table_labels):  
#     state_df, state_df_vc = print_val_data(t, label)
#     if(len(state_df_csv) == 0): state_df_csv = state_df.copy()
#     if(len(state_df_vc_csv) == 0): state_df_vc_csv = state_df_vc.copy()
#     for c in state_df.columns:
#         if c not in state_df_csv.columns:
#             state_df_csv[c] = state_df[c]
#     for c in state_df_vc.columns:
#         if c not in state_df_vc_csv.columns:
#             state_df_vc_csv[c] = state_df_vc[c]
        

# state_df_csv.to_csv("GCI_validation.csv")
# state_df_vc_csv.to_csv("VC_validation.csv")

feat$cat_ablation_045_075_w$msgs_100u$cnty$1gra


-0.417 & -0.545 & -0.664 & -0.542 & -0.542 \\
0.275 & 0.171 & 0.175 & 0.314 & 0.234 \\
0.003 & 0.0 & 0.0 & 0.0 \\
0.053 & 0.28 & 0.267 & 0.043 \\

feat$cat_ablation_045_07_w$msgs_100u$cnty$1gra
-0.417 & -0.545 & -0.664 & -0.542 & -0.542 \\
0.226 & 0.113 & 0.124 & 0.274 & 0.184 \\
0.003 & 0.0 & 0.0 & 0.0 \\
0.115 & 0.476 & 0.433 & 0.079 \\

feat$cat_ablation_045_08_w$msgs_100u$cnty$1gra
-0.417 & -0.545 & -0.664 & -0.542 & -0.542 \\
0.263 & 0.168 & 0.167 & 0.305 & 0.226 \\
0.003 & 0.0 & 0.0 & 0.0 \\
0.065 & 0.289 & 0.289 & 0.05 \\

feat$cat_ablation_04_075_w$msgs_100u$cnty$1gra
0.095 & 0.512 & 0.446 & 0.308 & 0.34 \\
0.273 & 0.504 & 0.531 & 0.552 & 0.465 \\
0.51 & 0.001 & 0.003 & 0.047 \\
0.055 & 0.001 & 0.0 & 0.0 \\

feat$cat_ablation_04_07_w$msgs_100u$cnty$1gra
0.095 & 0.512 & 0.446 & 0.308 & 0.34 \\
0.27 & 0.5 & 0.528 & 0.552 & 0.462 \\
0.51 & 0.001 & 0.003 & 0.047 \\
0.058 & 0.001 & 0.0 & 0.0 \\

feat$cat_ablation_04_08_w$msgs_100u$cnty$1gra
0.095 & 0.512 & 0.446 & 0.308 & 0.34 \\
0.

In [18]:
for t in PURIFICATION_NAMES:
    print(t)
    print_val_data(t, t)
    print()


feat$cat_purification_indv_0_01_w$msgs_100u$cnty$1gra


/home/shreyah/.local/lib/python3.6/site-packages/scipy/stats/stats.py:2497: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis, keepdims=True)
/home/shreyah/.local/lib/python3.6/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/shreyah/.local/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/shreyah/.local/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/shreyah/.local/lib/python3.6/site-packages/numpy/core/_methods.py:224: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


ValueError: zero-size array to reduction operation minimum which has no identity

### Looking at Collectivism/Individualism Scores: State level

In [ ]:
engine = sqlalchemy.create_engine(db_project)
county_scores = pd.read_sql("feat$cat_purification_015_w$msgs_100u$cnty$1gra", con=engine)

coll_scores_county = county_scores[county_scores["feat"] == "COLLECTIVISM"]
if(len(coll_scores_county) == 0):
    coll_scores_county = county_scores[county_scores["feat"] == "COLLECTIVIST"]
coll_scores_county = dict(zip(coll_scores_county.group_id, coll_scores_county.group_norm))

indv_scores_county = county_scores[county_scores["feat"] == "INDIVIDUALISM"]
if(len(indv_scores_county) == 0):
    indv_scores_county = county_scores[county_scores["feat"] == "INDIVIDUALIST"]
indv_scores_county = dict(zip(indv_scores_county.group_id, indv_scores_county.group_norm))

coll_scores_state = get_state_data(coll_scores_county)
indv_scores_state = get_state_data(indv_scores_county)

state_data = [living_data_state, religion_data, compatriotism_data, coll_scores_state, indv_scores_state, income_data_state]
state_labels = ["grandparents", "religion", "compatriotism", "collectivism", "individualism", "income"]

#make state df
states, state_data_processed = process_dicts(state_data)
state_df = pd.DataFrame(columns=state_labels)
for i in range(len(state_labels)):
    state_df[state_labels[i]] = zero_one_normalize(state_data_processed[i])
state_df.set_index(states, inplace=True)

print("Pairwise Pearson Correlations at the state level")
display(state_df.corr())

state_df.to_csv("validation.csv")


Pairwise Pearson Correlations at the state level


,grandparents,religion,compatriotism,collectivism,individualism,income
grandparents,1.000000,0.549095,0.317850,0.361601,-0.570941,-0.538756
religion,0.549095,1.000000,0.749607,0.410151,-0.659027,-0.413523
compatriotism,0.317850,0.749607,1.000000,0.467353,-0.515438,-0.180937
collectivism,0.361601,0.410151,0.467353,1.000000,-0.510319,-0.288011
individualism,-0.570941,-0.659027,-0.515438,-0.510319,1.000000,0.431243
income,-0.538756,-0.413523,-0.180937,-0.288011,0.431243,1.000000


### Looking at Collectivism/Individualism Scores: Community level

In [ ]:
_, mappings = get_community_mapping()

coll_scores_community = {}
indv_scores_community = {}

for fips, row in county_df.iterrows():
    try:
        c = mappings[fips]
        if(c not in coll_scores_community.keys()):
            coll_scores_community[c] = []
            indv_scores_community[c] = []
        coll_scores_community[c].append(coll_scores_county[fips])
        indv_scores_community[c].append(indv_scores_county[fips])
    except(Exception):
        # print("Couldn't map to community: ", fips)
        continue 
           
communities = list(coll_scores_community.keys())
for c in communities:
    if(len(coll_scores_community[c]) < 15):
        del coll_scores_community[c]
        del indv_scores_community[c]
    else:
        # print(c, " & ", len(coll_scores_community[c]), "\\\\")
        continue

community_df = pd.DataFrame()
communities = list(coll_scores_community.keys())
community_df["community"] = communities
community_df["collectivism"] = zero_one_normalize([np.mean(coll_scores_community[x]) for x in communities])
community_df["individualism"] = zero_one_normalize([np.mean(indv_scores_community[x]) for x in communities])

print("most individualistic communities")
print(community_df.sort_values(by="individualism", ascending=False).head(10))

community_df.to_csv("community_scores.csv")

NameError: name 'county_df' is not defined

### Vandello-Cohen Correlation

In [ ]:
vc_scores = pd.read_csv("GCI Data/vandello_cohen.csv")
vc_scores = dict(zip(vc_scores.State, vc_scores.Score))

state_data = [vc_scores, coll_scores_state, indv_scores_state]
state_labels = ["vandello-cohen", "collectivism", "individualism"]

#make state df
states, state_data_processed = process_dicts(state_data)
state_df = pd.DataFrame(columns=state_labels)
for i in range(len(state_labels)):
    state_df[state_labels[i]] = zero_one_normalize(state_data_processed[i])
state_df.set_index(states, inplace=True)

print("Pairwise Pearson Correlations at the state level")
display(state_df.corr())

state_df.to_csv("vc_scores.csv")



Pairwise Pearson Correlations at the state level


,vandello-cohen,collectivism,individualism
vandello-cohen,1.000000,0.275331,-0.415244
collectivism,0.275331,1.000000,-0.058544
individualism,-0.415244,-0.058544,1.000000


### Upload to DB

In [ ]:
county_data = [coll_scores_county, indv_scores_county]
county_labels = ["collectivism", "individualism"]

#make county df
fips_codes, state_data_processed = process_dicts(county_data)
county_df = pd.DataFrame(columns=county_labels)
for i in range(len(county_labels)):
    county_df[county_labels[i]] = (state_data_processed[i])
county_df.set_index(fips_codes, inplace=True)
county_df["cnty"] = fips_codes
# county_df["diff"] = zero_one_normalize(county_df["collectivism"]-county_df["individualism"])
county_df["diff"] = county_df["collectivism"]-county_df["individualism"]

print(county_df.head())
engine = sqlalchemy.create_engine(db_county)
county_df.to_sql("cnty_coll_indv_outcomes", con=engine, index=False, if_exists='replace', chunksize=50000)

      collectivism  individualism  cnty      diff
0         0.000598       0.001533     0 -0.000935
1001      0.000960       0.001439  1001 -0.000480
1003      0.000984       0.001639  1003 -0.000655
1005      0.001183       0.001437  1005 -0.000255
1007      0.001014       0.001510  1007 -0.000496


In [ ]:
db_indv_coll = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='individualism_collectivism', query={'read_default_file': '~/.my.cnf', 'charset':'utf8'})

engine = sqlalchemy.create_engine(db_indv_coll)
interpolations = pd.read_sql("ic2s2_interpolations", con=engine)

cnty = interpolations["cnty"]
diff_interpolated = interpolations["diff_interpolated"]

norm_cnty = []
norm_diff_interpolated = []
test = []
for i in range(len(cnty)):
    if(diff_interpolated[i] > 0 or diff_interpolated[i] <= 0):
        norm_cnty.append(cnty[i])
        norm_diff_interpolated.append(diff_interpolated[i])
        test.append(1)
norm_diff_interpolated = zero_one_normalize(norm_diff_interpolated)
print(len(norm_diff_interpolated))
normalized_diff = pd.DataFrame()
normalized_diff["cnty"] = norm_cnty
normalized_diff["diff"] = norm_diff_interpolated
normalized_diff["test"] = test

normalized_diff.to_sql("ic2s2_interpolations_normalized", con=engine, index=False, if_exists='replace', chunksize=50000)


/home/shreyah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  """Entry point for launching an IPython kernel.


3134


In [ ]:
county_df["diff"] = zscore(county_df["collectivism"]-county_df["individualism"])
diff = county_df["diff"]
reg_counties = county_df["cnty"].values

interpolations = pd.read_sql("ic2s2_interpolations", con=engine)
interpolations.dropna(inplace=True)
interpolations = interpolations[~interpolations["cnty"].isin(reg_counties)]
diff_interpolated = zscore(interpolations["diff_interpolated"])
interpolations["diff_interpolated"] = diff_interpolated
interpolated_counties = interpolations["cnty"].values

print(len(reg_counties), len(interpolated_counties))
print(len(diff_interpolated), "max: ", max(diff_interpolated), "min: ", min(diff_interpolated))
print(len(diff), "max: ", max(diff), "min: ", min(diff))
all_counties = list(set(list(reg_counties) + list(interpolated_counties)))
print(len(all_counties))


final_interpolations = pd.DataFrame()
final_cnty = []
final_diff = []
added = 0
for c in all_counties:
    if(c in reg_counties):
        final_cnty.append(c)
        final_diff.append(county_df[county_df["cnty"] == c]["diff"].values[0])
        added += 1
    elif(c in interpolated_counties):
        final_cnty.append(c)
        final_diff.append(interpolations[interpolations["cnty"] == c]["diff_interpolated"].values[0])
        added += 1
    else:
        continue
final_interpolations["cnty"] = final_cnty
final_interpolations["diff"] = final_diff
final_interpolations.dropna(inplace=True)
final_interpolations["diff"] = zero_one_normalize(final_interpolations["diff"])
final_interpolations.to_sql("ic2s2_interpolations_final", con=engine, index=False, if_exists='replace', chunksize=50000)



2042 1095
1095 max:  6.249860840375836 min:  -3.022431632005638
2042 max:  4.6184434211135335 min:  -5.39558067278122
3137


In [ ]:
len(final_interpolations)

3137